In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Using TensorFlow backend.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [5]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [6]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [7]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [10]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00126435 -0.00246562  0.00613835 ... -0.00130221  0.00310901
  -0.00611183]
 [ 0.00343964  0.00111127  0.00154763 ...  0.00136155  0.00059431
  -0.00309728]
 [ 0.0023258   0.00353537 -0.00394323 ...  0.00164968  0.00136141
  -0.00024369]
 ...
 [ 0.00588775  0.00304331  0.01098906 ... -0.00301385 -0.00045937
   0.00308184]
 [ 0.00678268 -0.00260056  0.0067612  ... -0.00393005 -0.00618262
   0.00327342]
 [ 0.00766157 -0.00128457  0.00424271 ...  0.00297033 -0.00718161
   0.00317423]], shape=(100, 65), dtype=float32)


In [15]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.26434735e-03 -2.46562157e-03  6.13834755e-03  4.13949648e-03
 -1.90720311e-03 -1.78153999e-03  3.66736250e-03 -2.30215909e-03
  1.23395654e-03  1.82087393e-03  3.01828608e-03 -3.77662620e-03
 -3.49510764e-03 -9.16348072e-04  3.52856726e-03  4.20967070e-03
  2.36731675e-03  5.44145424e-03  3.89296398e-03  3.37738474e-03
 -1.59334834e-03 -4.92470060e-03 -4.64800559e-03  1.57124805e-03
  1.68159138e-04 -2.92480364e-03 -1.65830285e-03 -7.97080458e-04
  4.27805819e-03 -6.60633855e-03  8.69399111e-04  3.59436846e-04
  5.46055241e-03 -2.56839627e-03  4.92730644e-03  4.08709189e-03
 -1.51492772e-04  3.81002389e-03  3.10964626e-03 -4.45011538e-03
 -1.19009055e-05  1.16514438e-03 -1.78032659e-03  7.57408736e-04
  5.87908900e-04  9.06243338e-04  4.34236880e-03 -5.17859263e-03
 -1.56722439e-03  1.88105996e-03 -4.01081517e-03 -2.60750158e-03
  3.42227286e-04  1.75353431e-03 -5.14705293e-03  2.24927091e-03
  3.64357792e-03  5.57088992e-04  3.02810152e-03  3.44072212e-03
 -2.0077072

In [16]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"Fz&VMDRW.y;BIwKWDUhJabqbKOztY?R!oE&oD.PzR'?F'AjEv:bFw&ROYUYyq3uAJMB3K-G:ipOR \nrTin.:pMhlULHGy;I i!$3"

In [17]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'yx.yikJS\nupfVAlwtbi&TXpLu.q33mD\nMzucebkwGRmjX WM!uQHpPwKmeqQxWRnEEpdCUlZQy  imZAakypZumYvBWnITt ,f&$'

In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [19]:
model.compile(optimizer='adam', loss=loss)

In [21]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 11s 64ms/step - loss: 2.6158
Epoch 2/50
172/172 [==============================] - 11s 64ms/step - loss: 1.9120
Epoch 3/50
172/172 [==============================] - 11s 65ms/step - loss: 1.6548
Epoch 4/50
172/172 [==============================] - 11s 66ms/step - loss: 1.5147
Epoch 5/50
172/172 [==============================] - 11s 67ms/step - loss: 1.4320
Epoch 6/50
172/172 [==============================] - 12s 67ms/step - loss: 1.3742
Epoch 7/50
172/172 [==============================] - 12s 68ms/step - loss: 1.3283
Epoch 8/50
172/172 [==============================] - 12s 68ms/step - loss: 1.2882
Epoch 9/50
172/172 [==============================] - 12s 69ms/step - loss: 1.2528
Epoch 10/50
172/172 [==============================] - 12s 69ms/step - loss: 1.2186
Epoch 11/50
172/172 [==============================] - 12s 70ms/step - loss: 1.1824
Epoch 12/50
172/172 [==============================] - 12s 70ms/step - loss: 1.1468
E

In [22]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [23]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [25]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [26]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeous lips in the morning: make thee
Deep have a think of both. The king is the beast.

CLARENCE:
Erron is cold entraily friendon from him: let him all pluck him years:
For you and here have strictue for his shame,
As vain. O player the queen;
Your king my blood with tears, I can afflict me
To assure me from thy mother's heavy weps!
Thy thy beave loath too much before
A vessage had,
That tigers smollow of his country's good
As that the lutely shall star nature sit and wail;
And let him shall be here made to your flesh and blood, s
Yet that Prospero for the next womb,
Then were her louding heart to thank
The silly want yourself we are but Clarence.

QUEEN ELIZABETH:
I have no more; what he's most love say:
You must to London on a subject's love,
Tendering the precipt cloupportions.

GLOUCESTER
